In [110]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys

In [111]:
SCC = pd.read_csv("../../data/SCC.csv")
Louv = pd.read_csv("../../data/Louv.csv")
LabelProp = pd.read_csv("../../data/LabelProp.csv")
Leiden = pd.read_csv("../../data/Leiden.csv")
nodes = pd.read_csv("../../data/nodes.csv")

In [112]:
def community_det_acc(df1, df2):
    df1['Node_ID:ID'] = [int(i.strip('"')) for i in df1['Name'].tolist()]
    df1.drop(columns=['Name'], inplace=True)

    result_df = pd.merge(df1, df2, on='Node_ID:ID', how='inner')

    percentage_breakdown = result_df.groupby(['Community', 'Status:LABEL']).size().unstack(fill_value=0)
    
    percentage_breakdown_percentage = percentage_breakdown.div(percentage_breakdown.sum(axis=1), axis=0) * 100
    
    return percentage_breakdown_percentage

In [113]:
community_det_acc(SCC, nodes)

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
0,33.248599,39.368729,27.382672


In [114]:
community_det_acc(Louv, nodes).head()

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
269,28.461538,38.076923,33.461538
336,29.530201,13.422819,57.046980
368,44.714640,11.315136,43.970223
438,39.487179,39.487179,21.025641
1185,38.657407,49.845679,11.496914


In [115]:
community_det_acc(LabelProp, nodes).head()

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
1,28.804429,17.970971,53.224600
6,27.483772,33.986137,38.530091
27,32.258270,50.928753,16.812977
66,15.248227,35.106383,49.645390
78,7.936508,50.793651,41.269841


In [116]:
community_det_acc(Leiden, nodes).head()

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
0,21.252796,41.387025,37.360179
1,29.411765,33.564014,37.024221
2,19.020173,40.634006,40.345821
3,32.459016,37.049180,30.491803
4,16.923077,36.923077,46.153846
